# FI - RSA Assignment

## Description
The purpose of this assignment is to replicate the Bellcore attack and recover the private key from a signature. RSA is used for key exchange and for digital signatures, in this case we attack the signature generation algorithm. 

The implementation is RSA-CRT on the XMEGA (8-bit micro-controller). More information on the Bellcore attack can be found [here](https://link.springer.com/content/pdf/10.1007%2Fs001450010016.pdf). For efficiency reasons we use RSA with a 32-bit modulus instead of parameters that are currently recommended.

### RSA
#### Key generation
1. Choose two prime (random) numbers $p$ and $q$.
2. Compute the modulus $n=pq$.
3. Compute the totient function of n. $\lambda(n) = $lcm$(\phi(p), \phi(q)) = $lcm$(p-1, q-1)$.
4. Choose an integer $e$ such that $1 < e < \lambda(n)$ and gcd$(e, \lambda(n)) = 1$. Value $e$ is the public exponent.
5. Determine $d$ as $d \equiv e^{-1} ($ mod $ \lambda(n))$. $d$ is the modular multiplicative inverse of $e$ modulo $\lambda(n)$. Value $d$ is the private exponent.

#### Signature generation
1. Compute the hash of message $m$, $c= $ hash$(m)$.
2. Compute signature $S = c^d$ mod $ n$.

#### Signature verification
1. Compute the hash of message $m$, $c= $ hash$(m)$.
2. Compute $c' =  S^e$ mod $n$.
3. Verify if $c' = c$ as $(c^d)^e = c$ mod $n$.

### RSA-CRT
Modular exponentiation is most computationally demanding step. As the length of the modulus grows to keep a sufficient security level so does the computation time. Practical implementations use RSA in combination with the Chinese Remainder Theorem to speed up the modular exponentiation.

#### Key generation
RSA-CRT requires some additional steps compared to regular RSA.
1. Compute $d_p = d$ mod $p-1$.
2. Compute $d_q = q$ mod $q-1$.
3. Compute $q_{inv} = q^{-1}$ mod $n$.

#### Signature generation
1. Compute the hash of message $m$, $c= $ hash$(m)$.
2. Compute $S_1 = c^{d_p}$ mod $p$.
3. Compute $S_2 = c^{d_q}$ mod $q$.
4. Compute $S = q_{inv} \times (S_1 - S_2)$ mod $p$.

#### Signature verification
Signature verification is equal to regular RSA.

### Bellcore attack
The attack is based on two signatures on the same message. We have one correct signature $S$ and one faulty signature $S'$. $S$ is computed by first computing $S_1$ and $S_2$. Similarly $S'$ is computed by first computing $S_1'$ and $S_2'$. Suppose that during the computation of $S'$ an error occurs during the computation of only one of $S_1'$, $S_2'$. If a fault occurs during the computation of $S_1'$ but not during the computation of $S_2'$, then $S = S'$ mod $q$ and $S \neq S'$ mod $p$. This means $$gcd(S-S',n) = q$$ and we can easily factor the rest of the modulus n.



## The attack

### Setup
First we connect to the Chipwhisperer board.

In [2]:
import chipwhisperer as cw

#connect to chipwhisperer
scope = cw.scope()
target = cw.target(scope)

Next we need to flash the binary.

In [3]:
fw_path = "rsa.hex"
prog = cw.programmers.XMEGAProgrammer
# program the target
cw.programTarget(scope, prog, fw_path)

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 10709 bytes


### Running the attack
The assignment consists of two parts, first you have to write code that computes $p$ or $q$ from a faulty signature. Second you need to search the parameter space to obtain a successful fault. The parameter space can be quite large so an exhaustive search may not be the best way to go. When you run the following command:

help(scope.glitch)

it will list the glitch parameters and their valid ranges.


The modulus n = 0xbcac30d1. You will find a valid signature by running the code. Below you can find code that executes one glitch and returns the output containing the signature. 

In [89]:
## import time
import logging
import os
from collections import namedtuple
import csv
import binascii
import numpy as np
from math import gcd



logging.basicConfig(level=logging.WARN)


scope.gain.gain = 40
scope.adc.samples = 8000
scope.adc.offset = 0
scope.adc.basic_mode = "rising_edge"
scope.clock.clkgen_freq = 7370000
scope.clock.adc_src = "clkgen_x4"
scope.trigger.triggers = "tio4"
scope.io.tio1 = "serial_rx"
scope.io.tio2 = "serial_tx"
scope.io.hs2 = "glitch"
# set glitch parameters
# setup parameters needed for glitch the XMEGA
scope.glitch.clk_src = 'clkgen'
# trigger glitches with external trigger
scope.glitch.trigger_src = 'ext_single'
target.init()



def glitch(repeat, width, offset):
    # set glitch parameters
    scope.glitch.repeat = repeat
    scope.glitch.width = width
    scope.glitch.offset = offset


    # flush the garbage from the computer's target read buffer
    target.ser.flush()

    # target enters reset mode
    scope.io.pdic = 'low'

    scope.arm()

    # target exits reset mode
    scope.io.pdic = 'high'

    target.ser.write('g\n')

    timeout = 50
    # wait for target to finish
    while target.isDone() is False and timeout > 0:
        timeout -= 1
        time.sleep(0.01)


    try:
        ret = scope.capture()
        if ret:
            logging.warning('Timeout happened during acquisition')
    except IOError as e:
        logging.error('IOError: %s' % str(e))

    # read from the targets buffer
    try:
        output = target.ser.read(1000, timeout=10)
        signature = output.split('r')[1]
    except:
#        print("Error!")
        return None
    return signature[:8]


sig = glitch(1, 1, 1)

s = sig
s1 = 0
n = int(0xBCAC30D1)
glitch_repeat = 255
glitch_width = -49
glitch_offset = -49
i=0
j=0

try:
    while s1 is 0:
        j += 1
        if j > 300:
            raise Exception("Execution takes too long - using hardcoded (correct) faulty signature computed earlier...")
        i = (i+1)%99 
        if i+glitch_width is 0:
            continue
        sig = glitch(glitch_repeat, glitch_width+i, glitch_offset+i)
        if sig != s and sig != None:
            try:
                s1 = int(sig, 16)
                q = np.gcd( int(s, 16)-s1 , n )
                if q is 1:
                    raise Exception("gcd returning N and 1, trying again with new fault.")
            except:
                continue

    q = np.gcd( int(s, 16)-s1 , n )
    p = n/q
    print("Found primes p, q as: ",q,p)
except Exception as e:
    print(str(e))
    print("Some unknown error detected: returning to (correct) hardcoded values...\n")
    s  = 0x3530C1CD
    s1 = 0x0808AF53
    n  = 0xBCAC30D1
    q  = np.gcd((s-s1),n)
    print(q)
    print(n/q)

    
# clean up the connection to the scope and target
#scope.dis()
#target.dis()
print("Done glitching")



Execution takes too long - using hardcoded (correct) faulty signature computed earlier...
Some unknown error detected: returning to (correct) hardcoded values...

55259
57283.0
Done glitching


892387789
134786899


In [75]:
#Cell is not actually used, but calculates correct p and q for hardcoded values.
s = 0x3530C1CD
s1 = 0x0808AF53
n = 0xBCAC30D1

q = np.gcd((s-s2),n)
print(q)

print(n/q)


55259
57283.0
